<a href="https://colab.research.google.com/github/Nyandala/Zindi-UmojaHack-South-Africa-Yassir-ETA-Prediction-Challenge/blob/main/Catboost%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package imports

In [1]:
import sys
!{sys.executable} -m pip install scikit-learn catboost

     |████████████████████████████████| 66.3MB 57kB/s 


In [2]:
import pandas as pd
import numpy as np
from math import sqrt
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Data imports

In [4]:
train = pd.read_csv('/content/drive/My Drive/Umoja hackathon/Train (1).csv')
test = pd.read_csv('/content/drive/My Drive/Umoja hackathon/Test (1).csv')
ss = pd.read_csv('/content/drive/My Drive/Umoja hackathon/SampleSubmission (2).csv')
weather = pd.read_csv('/content/drive/My Drive/Umoja hackathon/Weather.csv')

# Data Preprocessing and Feature Engineering

In [5]:
train['Timestamp'] = train['Timestamp'].str.split('T')
train['Timestamp'] =train['Timestamp'][1][0]
test['Timestamp'] = test['Timestamp'].str.split('T')
test['Timestamp'] =test['Timestamp'][1][0]

In [6]:
train = train.set_index('Timestamp').join(weather.set_index('date'), how='inner').reset_index()
test = test.set_index('Timestamp').join(weather.set_index('date'), how='inner').reset_index()

In [7]:
train['ETA'] = train.pop('ETA')

train.head()

,index,ID,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,dewpoint_2m_temperature,maximum_2m_air_temperature,mean_2m_air_temperature,mean_sea_level_pressure,minimum_2m_air_temperature,surface_pressure,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m,ETA
0,2019-12-10,000FLWA8,3.258,36.777,3.003,36.718,39627,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,2784
1,2019-12-10,000RGOAM,3.087,36.707,3.081,36.727,3918,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,576
2,2019-12-10,001QSGIH,3.144,36.739,3.088,36.742,7265,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,526
3,2019-12-10,002ACV6R,3.239,36.784,3.054,36.763,23350,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,3130
4,2019-12-10,0039Y7A8,2.912,36.707,3.207,36.698,36613,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,2138


In [8]:
train1 = train.copy()
train1.drop(columns = ['ID','index'], axis = 1, inplace = True)
train1.head()

,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,dewpoint_2m_temperature,maximum_2m_air_temperature,mean_2m_air_temperature,mean_sea_level_pressure,minimum_2m_air_temperature,surface_pressure,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m,ETA
0,3.258,36.777,3.003,36.718,39627,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,2784
1,3.087,36.707,3.081,36.727,3918,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,576
2,3.144,36.739,3.088,36.742,7265,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,526
3,3.239,36.784,3.054,36.763,23350,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,3130
4,2.912,36.707,3.207,36.698,36613,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,2138


In [9]:
test1 = test.copy()
test1.drop(columns = ['ID','index'], axis = 1, inplace = True)
test1.head()

,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,dewpoint_2m_temperature,maximum_2m_air_temperature,mean_2m_air_temperature,mean_sea_level_pressure,minimum_2m_air_temperature,surface_pressure,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m
0,2.981,36.688,2.978,36.754,17549,282.767426,293.182098,287.179047,102420.648438,283.15921,101367.804688,-1.860000e-09,-0.946755,0.58422
1,3.032,36.769,3.074,36.751,7532,282.767426,293.182098,287.179047,102420.648438,283.15921,101367.804688,-1.860000e-09,-0.946755,0.58422
2,3.035,36.711,3.010,36.758,10194,282.767426,293.182098,287.179047,102420.648438,283.15921,101367.804688,-1.860000e-09,-0.946755,0.58422
3,2.902,36.738,3.208,36.698,32768,282.767426,293.182098,287.179047,102420.648438,283.15921,101367.804688,-1.860000e-09,-0.946755,0.58422
4,2.860,36.692,2.828,36.696,4513,282.767426,293.182098,287.179047,102420.648438,283.15921,101367.804688,-1.860000e-09,-0.946755,0.58422


In [10]:
X = train1.iloc[:, :-1].values
y = train1.iloc[:, -1].values
X_test = test1.iloc[:,:].values

In [11]:
train1

,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,dewpoint_2m_temperature,maximum_2m_air_temperature,mean_2m_air_temperature,mean_sea_level_pressure,minimum_2m_air_temperature,surface_pressure,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m,ETA
0,3.258,36.777,3.003,36.718,39627,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,2784
1,3.087,36.707,3.081,36.727,3918,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,576
2,3.144,36.739,3.088,36.742,7265,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,526
3,3.239,36.784,3.054,36.763,23350,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,3130
4,2.912,36.707,3.207,36.698,36613,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,2138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83919,3.121,36.743,3.032,36.770,10321,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,664
83920,3.024,36.749,2.951,36.735,10791,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,1056
83921,3.189,36.721,3.186,36.721,1139,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,296
83922,3.046,36.738,2.986,36.766,10637,282.286621,289.168976,286.988831,102472.148438,283.798737,101401.242188,0.001322,2.550383,-1.544611,895


In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [13]:
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
X_test_scaled = X_test.copy()

sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_val_scaled = sc.transform(X_val)
X_test_scaled = sc.transform(X_test)

# Modelling

In [14]:
cb_model = CatBoostRegressor(iterations=10750,
                             learning_rate=0.02,
                             depth=12,
                             eval_metric='RMSE',
                             random_seed = 0,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)
cb_model.fit(X_train, y_train, eval_set=(X_val,y_val), use_best_model=True,verbose=True)
y_pred_val = cb_model.predict(X_val)
cb_model_rmse = sqrt(metrics.mean_squared_error(y_val, y_pred_val))
cb_model_rmse

0:	learn: 553.9349964	test: 556.3759522	best: 556.3759522 (0)	total: 207ms	remaining: 37m 8s
75:	learn: 239.7219116	test: 242.5725674	best: 242.5725674 (75)	total: 6.85s	remaining: 16m 2s
150:	learn: 194.1701405	test: 197.9418447	best: 197.9418447 (150)	total: 13.6s	remaining: 15m 51s
225:	learn: 180.9946098	test: 185.2127848	best: 185.2127848 (225)	total: 20.3s	remaining: 15m 45s
300:	learn: 173.5618612	test: 178.3267320	best: 178.3267320 (300)	total: 26.8s	remaining: 15m 31s
375:	learn: 168.4904332	test: 173.7105545	best: 173.7105545 (375)	total: 33.4s	remaining: 15m 21s
450:	learn: 164.4954947	test: 170.4157126	best: 170.4157126 (450)	total: 40s	remaining: 15m 13s
525:	learn: 161.0299144	test: 167.6510330	best: 167.6510330 (525)	total: 46.5s	remaining: 15m 4s
600:	learn: 158.0653582	test: 165.4030344	best: 165.4030344 (600)	total: 53.1s	remaining: 14m 56s
675:	learn: 155.2390679	test: 163.3337667	best: 163.3337667 (675)	total: 59.6s	remaining: 14m 48s
750:	learn: 152.7698323	test: 1

135.56377354708164

In [15]:
pred_cb = cb_model.predict(test1)

# Submission File

In [ ]:
ss_final = ss.copy()
sub = pd.DataFrame({'ID':ss_final['ID'], 'ETA': pred_cb})
sub.to_csv('basic_cb.csv', index = False)